In [3]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras import applications, layers, losses, optimizers, Model
from tensorflow.keras.models import Sequential
import keras.backend as K
from sklearn.model_selection import train_test_split
from textwrap import wrap
from transformers import ResNetConfig, ResNetModel
from transformers import AutoImageProcessor, TFResNetForImageClassification
import tensorflow as tf



In [4]:
pretrained_model = TFResNetForImageClassification.from_pretrained("microsoft/resnet-50")

All model checkpoint layers were used when initializing TFResNetForImageClassification.

All the layers of TFResNetForImageClassification were initialized from the model checkpoint at microsoft/resnet-50.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFResNetForImageClassification for predictions without further training.


In [26]:
PATH = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/'
PATH_TO_IMG = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train_images/'
PATH_TO_TEST = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/test_images/'

In [22]:
checkpoint_filepath = '/workspaces/Shopee-Price-Match-Guarantee/30_results/HG_ResNet50/checkpoints/'


In [30]:
class_names = train_ds.class_names
num_classes = len(train_ds.class_names)


# import the ResNet50 model from keras

resnet = Sequential()
pretrained_model = applications.ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3), # change from 256 to 224
    pooling='avg',
    classes=num_classes,
    classifier_activation='softmax',
)

for layer in pretrained_model.layers:
    layer.trainable = False

resnet.add(pretrained_model)
# add batch normalization layer
resnet.add(BatchNormalization())
# flatten the output
resnet.add(Flatten())
# add dropout layer
resnet.add(Dropout(0.5))
# add linear layer for reduction
resnet.add(Dense(512, activation='linear'))
# add classification layer
resnet.add(Dense(num_classes, activation='softmax'))

In [ ]:
epochs=5 
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Hugging faces

In [13]:
from transformers import TFResNetForImageClassification
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from datasets import load_dataset

class_names = train_ds.class_names
num_classes = len(train_ds.class_names)

# import the ResNet50 model from transformers
pretrained_model = TFResNetForImageClassification.from_pretrained("microsoft/resnet-50")





All model checkpoint layers were used when initializing TFResNetForImageClassification.

All the layers of TFResNetForImageClassification were initialized from the model checkpoint at microsoft/resnet-50.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFResNetForImageClassification for predictions without further training.


In [14]:
image = load_dataset(PATH_TO_IMG) # load the dataset

Resolving data files:   0%|          | 0/32411 [00:00<?, ?it/s]

Found cached dataset imagefolder (/Users/alisa/.cache/huggingface/datasets/imagefolder/train_images-82e4eb5df6582b4a/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = TFResNetForImageClassification.from_pretrained("microsoft/resnet-50")

inputs = image_processor(image["train"]["image"], return_tensors="tf")
# inputs = image
logits = model(**inputs).logits
# model predicts one of the 1000 ImageNet classes
predicted_label = int(tf.math.argmax(logits, axis=-1))
print(model.config.id2label[predicted_label])

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
All model checkpoint layers were used when initializing TFResNetForImageClassification.

All the layers of TFResNetForImageClassification were initialized from the model checkpoint at microsoft/resnet-50.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFResNetForImageClassification for predictions without further training.


In [60]:
image["train"]["image"][0]

KeyboardInterrupt: 

In [21]:
from transformers import TFResNetForImageClassification
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential

class_names = train_ds.class_names
num_classes = len(train_ds.class_names)

# import the ResNet50 model from transformers
pretrained_model = TFResNetForImageClassification.from_pretrained("microsoft/resnet-50")

# remove the classification head
pretrained_model = tf.keras.Model(inputs=pretrained_model.input, outputs=pretrained_model.layers[-3].output)

resnet = Sequential()
resnet.add(pretrained_model)

# add batch normalization layer
resnet.add(BatchNormalization())
# flatten the output
resnet.add(Flatten())
# add dropout layer
resnet.add(Dropout(0.5))
# add linear layer for reduction
resnet.add(Dense(512, activation='linear'))
# add classification layer
resnet.add(Dense(num_classes, activation='softmax'))

epochs = 5

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

resnet.compile(optimizer='adam', 
                loss='categorical_crossentropy',
                metrics=[get_f1])

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    # monitor='val_loss',
    mode='max',
    save_best_only=True,
    verbose=1,
    save_freq='epoch')

# load checkpoint if exist
# resnet.load_weights(checkpoint_filepath)

resnet.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[model_checkpoint_callback])


All model checkpoint layers were used when initializing TFResNetForImageClassification.

All the layers of TFResNetForImageClassification were initialized from the model checkpoint at microsoft/resnet-50.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFResNetForImageClassification for predictions without further training.


AttributeError: Layer tf_res_net_for_image_classification_2 is not connected, no input to return.